### Establishing Connection to the Database

In [1]:
#connecting to the database
import psycopg2

username="deepankaracharyya"
passwd="password"

conn = psycopg2.connect(database="GeneticSequencing", user = username, password = passwd, host = "127.0.0.1", port = "5432")
cursor = conn.cursor()

## Processing the file and entering the data

In [2]:
from geneInfo import *
import re

#### Opening the file pointers

In [3]:
geneFilePointer = open("resources/sample_01.ena")
proteinFilePointer = open("output/proteins.txt","w")

#### Defing the input queries

In [4]:
insert_query_01="INSERT INTO GeneticData VALUES (%s,%s, %s,%s, %s)"
insert_query_02="INSERT INTO AminoAcidData VALUES (%s,%s, %s,%s, %s)"
insert_query_03="INSERT INTO AminoAcidSequencing_01 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

#### Defining the fetching queries

In [5]:
fetch_query_01="SELECT * from GeneticData"
fetch_query_02="SELECT * from AminoAcidData"
fetch_query_03="SELECT * from AminoAcidSequencing_01"

#### Global Variables

In [6]:
geneName=''
geneNumber=0
speciesName=''
geneSequence=''
aminoAcidSequence=''
remove_symbols1="["+"\\n:>"+"]"

### Function for counting the occurances of a particular Amino Acid in a Gene:

In [7]:
amino_acids=["I","L","V","F","M","C","A","G","P","T","S","Y","W","Q","N","H","E","D","K","R",]
amino_acid_index_dic={"I":0,"L":1,"V":2,"F":3,"M":4,"C":5,"A":6,"G":7,"P":8,"T":9,"S":10,"Y":11,"W":12,"Q":13,"N":14,"H":15,"E":16,"D":17,"K":18,"R":19}

In [8]:
def count_amino_acids(sequence):
    amino_acid_counts=[0 for i in range(len(amino_acids))]
    for i in range(len(amino_acid_counts)):
        amino_acid_counts[i]=sequence.count(amino_acids[i])
    return amino_acid_counts

#### The Processing

In [9]:
for eachLine in (geneFilePointer):
    if '>' in eachLine:
        if geneNumber!=0:
            #insert into the database
            cursor.execute(insert_query_01,(geneNumber,geneName,geneSequence,len(geneSequence),"Good"))
            cursor.execute(insert_query_02,(geneNumber,geneName,aminoAcidSequence,len(aminoAcidSequence),"Good"))
            count_list=count_amino_acids(aminoAcidSequence)
            cursor.execute(insert_query_03,(geneNumber,geneName,count_list[0],count_list[1],count_list[2],count_list[3],count_list[4],count_list[5],count_list[6],count_list[7],count_list[8],count_list[9],count_list[10],count_list[11],count_list[12],count_list[13],count_list[14],count_list[15],count_list[16],count_list[17],count_list[18],count_list[19]))
            #commit the changes
            conn.commit()
        #flush the geneSequence
        geneSequence=''
        #flush the amino acid sequence
        aminoAcidSequence=''
        #contine reading
        eachLine=re.sub(remove_symbols1,"",eachLine)
        line=re.split("\|", eachLine)
        speciesName=line[0]
        geneNumber=line[1]
        geneName=line[2]
        #continue
        continue
    
    else :
        #going  into a while loop
        i=0
        while i < len(eachLine)-1:
            condon=eachLine[i:i+3]
            #adding to the geneSequence
            geneSequence=geneSequence+condon
            condon_con=genetic_code_table[condon]
            if(condon_con=='*'):
                break
            #converting and adding to the Amino Acid Sequence
            aminoAcidSequence=aminoAcidSequence + condon_con
            i=i+3   
        
        

#### Closing the file pointers

In [10]:
geneFilePointer.close()
proteinFilePointer.close()

### Verifying the Inserted Data:

In [11]:
cursor.execute(fetch_query_01)
all=cursor.fetchall()
print(all)

[(2, 'thrA', 'atgcgagtgttgaagttcggcggtacatcagtggcaaatgcagaacgttttctgcgtgttgccgatattctggaaagcaatgccaggcaggggcaggtggccaccgtcctctctgcccccgccaaaatcaccaaccacctggtggcgatgattgaaaaaaccattagcggccaggatgctttacccaatatcagcgatgccgaacgtatttttgccgaacttttgacgggactcgccgccgcccagccggggttcccgctggcgcaattgaaaactttcgtcgatcaggaatttgcccaaataaaacatgtcctgcatggcattagtttgttggggcagtgcccggatagcatcaacgctgcgctgatttgccgtggcgagaaaatgtcgatcgccattatggccggcgtattagaagcgcgcggtcacaacgttactgttatcgatccggtcgaaaaactgctggcagtggggcattacctcgaatctaccgtcgatattgctgagtccacccgccgtattgcggcaagccgcattccggctgatcacatggtgctgatggcaggtttcaccgccggtaatgaaaaaggcgaactggtggtgcttggacgcaacggttccgactactctgctgcggtgctggctgcctgtttacgcgccgattgttgcgagatttggacggacgttgacggggtctatacctgcgacccgcgtcaggtgcccgatgcgaggttgttgaagtcgatgtcctaccaggaagcgatggagctttcctacttcggcgctaaagttcttcacccccgcaccattacccccatcgcccagttccagatcccttgcctgattaaaaataccggaaatcctcaagcaccaggtacgctcattggtgccagccgtgatgaagacgaattaccggtcaagggcatttccaatctgaataacatggcaatgttcagcgtttctggtccggggatgaaagggatggt

In [12]:
cursor.execute(fetch_query_02)
all=cursor.fetchall()
print(all)

[(2, 'thrA', 'MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNHLVAMIEKTISGQDALPNISDAERIFAELLTGLAAAQPGFPLAQLKTFVDQEFAQIKHVLHGISLLGQCPDSINAALICRGEKMSIAIMAGVLEARGHNVTVIDPVEKLLAVGHYLESTVDIAESTRRIAASRIPADHMVLMAGFTAGNEKGELVVLGRNGSDYSAAVLAACLRADCCEIWTDVDGVYTCDPRQVPDARLLKSMSYQEAMELSYFGAKVLHPRTITPIAQFQIPCLIKNTGNPQAPGTLIGASRDEDELPVKGISNLNNMAMFSVSGPGMKGMVGMAARVFAAMSRARISVVLITQSSSEYSISFCVPQSDCVRAERAMQEEFYLELKEGLLEPLAVTERLAIISVVGDGMRTLRGISAKFFAALARANINIVAIAQGSSERSISVVVNNDDATTGVRVTHQMLFNTDQVIEVFVIGVGGVGGALLEQLKRQQSWLKNKHIDLRVCGVANSKALLTNVHGLNLENWQEELAQAKEPFNLGRLIRLVKEYHLLNPVIVDCTSSQAVADQYADFLREGFHVVTPNKKANTSSMDYYHQLRYAAEKSRRKFLYDTNVGAGLPVIENLQNLLNAGDELMKFSGILSGSLSYIFGKLDEGMSFSEATTLAREMGYTEPDPRDDLSGMDVARKLLILARETGRELELADIEIEPVLPAEFNAEGDVAAFMANLSQLDDLFAARVAKARDEGKVLRYVGNIDEDGVCRVKIAEVDGNDPLFKVKNGENALAFYSHYYQPLPLVLRGYGAGNDVTAAGVFADLLRTLSWKLGV', 820, 'Good'), (3, 'thrB', 'MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSLNNLGRFADKLPSEPRENIVYQCWERFCQELGKQIPVAMTLEKNMPIGSGLGSSACSVVAALMAMNEHCGKPLNDTRLLALMGELEGRISG

In [13]:
cursor.execute(fetch_query_03)
all=cursor.fetchall()
print(all)

[(2, 'thrA', 46, 89, 69, 30, 23, 12, 91, 63, 29, 34, 51, 20, 4, 30, 38, 14, 53, 44, 34, 46), (3, 'thrB', 16, 31, 23, 10, 10, 9, 37, 27, 16, 9, 17, 8, 4, 14, 12, 6, 21, 13, 10, 17), (4, 'thrC', 17, 54, 27, 24, 8, 4, 47, 28, 23, 23, 19, 9, 3, 21, 19, 10, 29, 24, 22, 17), (5, 'yaaX', 3, 7, 6, 0, 2, 0, 7, 11, 9, 1, 3, 3, 6, 4, 2, 13, 2, 5, 7, 7)]


### Closing the connection

In [14]:
conn.commit()
conn.close()
print("\n Connection Closed!\n")


 Connection Closed!

